# Variation 8b. QUEΛ to SQL 

Warm up exercise: a string representation for QUEΛ queries

In [15]:
type StringRep[T] = String => Int => String

implicit object StringQUEΛ extends QUEΛ[StringRep]{
    // base types
    
    def bool(b: Boolean): StringRep[Boolean] = 
        _ => _ => b.toString
    
    def int(i: Int): StringRep[Int] = 
        _ => _ => i.toString
    
    def str(s: String): StringRep[String] = 
        _ => _ => s
    
    def >(i1: StringRep[Int], i2: StringRep[Int]): StringRep[Boolean] = 
        t => i => s"${i1(t)(i)} > ${i2(t)(i)}"
        
    // ADTs
    
    def tuple2[A, B](a: StringRep[A], b: StringRep[B]): StringRep[(A, B)] = 
        t => i => s"""(${a(t)(i)}, ${b(t)(i)})"""
    
    def none[A]: StringRep[Option[A]] = 
        _ => _ => "none"
    
    def some[A](a: StringRep[A]): StringRep[Option[A]] = 
        t => i => s"some(${a(t)(i)})"
    
    def exists[A](o: StringRep[A])(cond: StringRep[A] => StringRep[Boolean]): StringRep[Boolean] = 
        t => i => s"""exists(${o(t)(i)})(x$i => ${cond(_ => _ => s"x$i")(t)(i)})"""
    
    // Comprehensions
    
    def bind[A, B](q: StringRep[List[A]])(f: StringRep[A] => StringRep[List[B]]): StringRep[List[B]] = 
        t => i => 
            s"""|${t}bind(
                |${q(t+"    ")(i)}
                |${t}){ x$i => 
                |${f(_ => _ => s"x$i")(t+"    ")(i+1)}
                |${t}}""".stripMargin
    
    def pure[A](a: StringRep[A]): StringRep[A] = 
        t => i => t + s"pure(${a(t)(i)})"
    
    def where[A](cond: StringRep[Boolean])(q: StringRep[List[A]]): StringRep[List[A]] = 
        t => i => 
            s"""|${t}where(${cond(t)(i)})(
                |${q(t+"    ")(i)}
                |${t})""".stripMargin
    
    // Equality
    
    def ===[A](a1: StringRep[A], a2: StringRep[A]): StringRep[Boolean] = 
        t => i => s"===(${a1(t)(i)}, ${a2(t)(i)})"
}

defined type StringRep
defined object StringQUEΛ

In [16]:
object CityStringRep extends CityModel[StringRep]{
    def id(city: StringRep[City]): StringRep[Int] = 
        t => i => s"id(${city(t)(i)})"
    
    def name(city: StringRep[City]): StringRep[String] = 
        t => i => s"cityName(${city(t)(i)})"
    
    def country(city: StringRep[City]): StringRep[String] = 
        t => i => s"country(${city(t)(i)})"
    
    def population(city: StringRep[City]): StringRep[Int] = 
        t => i => s"population(${city(t)(i)})"
}

object CountryStringRep extends CountryModel[StringRep]{
    def code(country: StringRep[Country]): StringRep[String] = 
        t => i => s"code(${country(t)(i)})"
    
    def name(country: StringRep[Country]): StringRep[String] = 
        t => i => s"name(${country(t)(i)})"
    
    def capital(country: StringRep[Country]): StringRep[Int] = 
        t => i => s"capital(${country(t)(i)})"
}

implicit object WorldStringRep extends WorldModel[StringRep]{
    val Ci = CityStringRep
    val Co = CountryStringRep
    
    def countries: StringRep[List[Country]] = 
        t => _ => t + s"countries"
    
    def cities: StringRep[List[City]] = 
        t => _ => t + s"cities"
}

defined object CityStringRep
defined object CountryStringRep
defined object WorldStringRep

In [17]:
def test1[Repr[_]](implicit Q: QUEΛ[Repr], W: WorldModel[Repr]): Repr[List[(Country, (Country, Country))]] = 
    W.countries.flatMap{ c1 => 
        W.countries.flatMap{ c2 => 
            W.countries.flatMap{ c3 => 
                Q.pure((c1, (c2, c3): Repr[(Country, Country)]))
            }
        }
    }

defined function test1

In [18]:
def test2[Repr[_]](implicit Q: QUEΛ[Repr], W: WorldModel[Repr]): Repr[List[((Country, Country), Country)]] = 
    (W.countries.flatMap{ c1 => 
        W.countries.flatMap{ c2 => 
            Q.pure((c1, c2))
        }
    }).flatMap{ c1c2 => 
        W.countries.flatMap{ c3 => 
            Q.pure((c1c2, c3))
        }
    }

defined function test2

In [19]:
println(test1[StringRep].apply("")(0))
println(test2[StringRep].apply("")(0))

bind(
    countries
){ x0 => 
    bind(
        countries
    ){ x1 => 
        bind(
            countries
        ){ x2 => 
            pure((x0, (x1, x2)))
        }
    }
}
bind(
    bind(
        countries
    ){ x0 => 
        bind(
            countries
        ){ x1 => 
            pure((x0, x1))
        }
    }
){ x0 => 
    bind(
        countries
    ){ x1 => 
        pure((x0, x1))
    }
}


In [20]:
println(largeCapitals[StringRep].apply("")(0))

bind(
    countries
){ x0 => 
    bind(
        bind(
            bind(
                cities
            ){ x1 => 
                where(exists(capital(x0))(x2 => ===(x2, id(x1))))(
                    pure(x1)
                )
            }
        ){ x1 => 
            where(population(x1) > 8000000)(
                pure(x1)
            )
        }
    ){ x1 => 
        pure((cityName(x1), name(x0)))
    }
}


In [21]:
def largeCapitalsNormalized[Repr[_]](implicit Q: QUEΛ[Repr], W: WorldModel[Repr]): Repr[List[(String, String)]] = 
    W.countries.flatMap{ x0 => 
        W.cities.flatMap{ x1 => 
            Q.where(capital(x0).exists(x2 => x2 === id(x1)))(
                Q.where(population(x1) > Q.int(8000000))(
                    Q.pure((cityName(x1), name(x0)))
                )
            )
        }
    }

defined function largeCapitalsNormalized

In [22]:
println(largeCapitalsNormalized[StringRep].apply("")(0))

bind(
    countries
){ x0 => 
    bind(
        cities
    ){ x1 => 
        where(exists(capital(x0))(x2 => ===(x2, id(x1))))(
            where(population(x1) > 8000000)(
                pure((cityName(x1), name(x0)))
            )
        )
    }
}


In [23]:
largeCapitalsNormalized[WorldReader]
    .apply(realWorld)
    .value
    .timed

2368 millis


res22: List[(String, String)] = List(
  ("Jakarta", "Indonesia"),
  ("Ciudad de M\u00e9xico", "Mexico"),
  ("Moscow", "Russian Federation"),
  ("Seoul", "South Korea")
)

## SQL interpreter

In [24]:
sealed abstract class DoobieRep[T]{ val fragment: Fragment }
case class SQLVar[A](fragment: Fragment) extends DoobieRep[A]
case class SQLInt(fragment: Fragment) extends DoobieRep[Int]
case class SQLBoolean(fragment: Fragment) extends DoobieRep[Boolean]
case class SQLString(fragment: Fragment) extends DoobieRep[String]
case class SQLOption[A](rep: DoobieRep[A]) extends DoobieRep[Option[A]]{
    val fragment = rep.fragment
}
case class SQLTuple[A, B](
    fr1: Fragment, 
    fr2: Fragment) extends DoobieRep[(A, B)]{
    val fragment = fr1 ++ fr", " ++ fr2
}
case class SQLStatement[A](
    select: Fragment, 
    from: List[Fragment], 
    where: List[Fragment]
) extends DoobieRep[List[A]]{
    val fragment = 
        fr"select" ++ select ++ 
        fr"from" ++ from.combineAll ++ 
        fr"where" ++ where.combineAll
}

object DoobieRep{

    implicit object DoobieQUEΛ extends QUEΛ[DoobieRep]{
        // base types

        def bool(b: Boolean): DoobieRep[Boolean] = 
            SQLBoolean(Fragment.const(b.toString))

        def int(i: Int): DoobieRep[Int] = 
            SQLInt(Fragment.const(i.toString))

        def str(s: String): DoobieRep[String] = 
            SQLString(Fragment.const(s))

        def >(i1: DoobieRep[Int], i2: DoobieRep[Int]): DoobieRep[Boolean] = 
            SQLBoolean(i1.fragment ++ fr" > " ++ i2.fragment)

        // ADTs

        def tuple2[A, B](a: DoobieRep[A], b: DoobieRep[B]): DoobieRep[(A, B)] = 
            SQLTuple(a.fragment, b.fragment) 

        def none[A]: DoobieRep[Option[A]] = 
            ???

        def some[A](a: DoobieRep[A]): DoobieRep[Option[A]] = 
            ???

        def exists[A](o: DoobieRep[Option[A]])(cond: DoobieRep[A] => DoobieRep[Boolean]): DoobieRep[Boolean] = {
            val SQLOption(a) = o
            cond(a)
        }

        // Comprehensions

        def bind[A, B](q: DoobieRep[List[A]])(f: DoobieRep[A] => DoobieRep[List[B]]): DoobieRep[List[B]] = {
            val SQLStatement(select, from, where) = f(SQLVar(fr"x0"))
            SQLStatement(select, q.fragment :: from, where)
        }

        def pure[A](a: DoobieRep[A]): DoobieRep[List[A]] = 
            SQLStatement(a.fragment, List(), List())

        def where[A](cond: DoobieRep[Boolean])(q: DoobieRep[List[A]]): DoobieRep[List[A]] = {
            val SQLStatement(select, from, where) = q
            SQLStatement(select, from, cond.fragment :: where)
        }
        
        // Equality

        def ===[A](a1: DoobieRep[A], a2: DoobieRep[A]): DoobieRep[Boolean] = 
            SQLBoolean(a1.fragment ++ fr"=" ++ a2.fragment)
    }
}

defined class DoobieRep
defined class SQLVar
defined class SQLInt
defined class SQLBoolean
defined class SQLString
defined class SQLOption
defined class SQLTuple
defined class SQLStatement
defined object DoobieRep

In [28]:
object CityDoobieRep extends CityModel[DoobieRep]{
    def id(city: DoobieRep[City]): DoobieRep[Int] = 
        SQLInt(city.fragment ++ fr"id")
    
    def name(city: DoobieRep[City]): DoobieRep[String] = 
        SQLString(city.fragment ++ fr".name")
    
    def country(city: DoobieRep[City]): DoobieRep[String] = 
        SQLString(city.fragment ++ fr".country")
    
    def population(city: DoobieRep[City]): DoobieRep[Int] = 
        SQLInt(city.fragment ++ fr".population")
}

object CountryDoobieRep extends CountryModel[DoobieRep]{
    def code(country: DoobieRep[Country]): DoobieRep[String] = 
        SQLString(country.fragment ++ fr".code")
    
    def name(country: DoobieRep[Country]): DoobieRep[String] = 
        SQLString(country.fragment ++ fr".name")
    
    def capital(country: DoobieRep[Country]): DoobieRep[Option[Int]] = 
        SQLOption(SQLInt(country.fragment ++ fr".capital"))
}

implicit object WorldDoobieRep extends WorldModel[DoobieRep]{
    val Ci = CityDoobieRep
    val Co = CountryDoobieRep
    
    def countries: DoobieRep[List[Country]] = 
        SQLVar[List[Country]](fr"countries")
    
    def cities: DoobieRep[List[City]] = 
        SQLVar[List[City]](fr"cities")
}

defined object CityDoobieRep
defined object CountryDoobieRep
defined object WorldDoobieRep

In [30]:
largeCapitalsNormalized[DoobieRep].fragment
largeCapitals[DoobieRep].fragment

res29_0: Fragment = Fragment("select x0 .name ,  x0 .name from countries cities where x0 .capital = x0 id x0 .population  >  8000000 ")
res29_1: Fragment = Fragment("select x0 .name ,  x0 .name from countries select x0 from select x0 from cities where x0 .capital = x0 id where x0 .population  >  8000000 where ")